# imports


In [ ]:
from __future__ import annotations

import os
import sys
from dataclasses import dataclass, field
from typing import (
    Any,
    Callable,
    Dict,
    Iterable,
    List,
    Literal,
    Mapping,
    Optional,
    Tuple,
)

import lightning as L
import loguru
import numpy as np
import torch
import torch.nn.functional as F
from lightning.pytorch.loggers import CSVLogger, WandbLogger
from torch import Tensor, nn, optim

sys.path.insert(0, "../")
from src.utils.logger import create_logger
from src.utils.torchsummary import summary

In [2]:
logger = create_logger()

# data

prepare the dataset first


In [3]:
class Group:
    """
    Generic finite group defined by an explicit carrier set and a binary operation.

    This class validates the group axioms exhaustively:
        - Closure
        - Associativity
        - Identity
        - Inverses

    It also detects whether the group is Abelian (commutative).

    Notes
    -----
    - This implementation is intended for *finite* groups only.
    - Associativity checking is O(n^3) and should only be used for small sets.
    - Equality is handled for scalars, NumPy arrays, and Torch tensors.
    """

    def __init__(
        self,
        elements: List[int | float] | np.ndarray | Tensor,
        op: Callable,
        logger: Optional[loguru.Logger] = None,
        *args,
        **kwargs,
    ) -> None:
        """
        Parameters
        ----------
        elements : list | np.ndarray | torch.Tensor
            Explicit enumeration of group elements.
        op : Callable
            Binary operation defining the group law.
        logger : loguru.Logger, optional
            Logger instance for diagnostics.
        """
        self.elements = list(elements)
        self._op = op

        if logger is None:
            logger = create_logger()
        self.logger = logger

        self.identity: Any = None
        self._right_inverses: Dict[Any, Any] = {}
        self._left_inverses: Dict[Any, Any] = {}
        self.is_abelian: bool = False

        self._validate_group()

    def _equal(self, a, b) -> bool:
        """
        Robust equality comparison across supported element types.
        """
        if isinstance(a, Tensor) and isinstance(b, Tensor):
            return torch.equal(a, b)
        if isinstance(a, np.ndarray) and isinstance(b, np.ndarray):
            return np.array_equal(a, b)
        return a == b

    def _in_elements(self, x) -> bool:
        """
        Check whether an element belongs to the carrier set.
        """
        return any(self._equal(x, e) for e in self.elements)

    def _validate_group(self) -> None:
        """
        Validate all group axioms and compute derived properties.
        """
        self.logger.info("Validating group axioms")
        try:
            self._check_closure()
            self._check_associativity()
            self.identity = self._find_op_identity()
            self._find_op_inverses()
        except Exception as e:
            self.logger.error(
                f"Group valildation failed due to the following reason: {e}",
                exc_info=True,
            )
            raise e
        self.is_abelian = self._check_abelian()

        self.logger.success("Group validation successful")

    def _check_closure(self) -> None:
        """
        Verify closure of the operation over the carrier set.
        """
        for a in self.elements:
            for b in self.elements:
                result = self.op(a, b)
                if not self._in_elements(result):
                    raise ValueError(
                        f"Closure violated: {a} * {b} = {result} not in elements"
                    )

    def _check_associativity(self) -> None:
        """
        Verify associativity of the binary operation.
        """
        for a in self.elements:
            for b in self.elements:
                for c in self.elements:
                    left = self.op(self.op(a, b), c)
                    right = self.op(a, self.op(b, c))
                    if not self._equal(left, right):
                        raise ValueError(
                            f"Associativity violated: ({a} * {b}) * {c} != {a} * ({b} * {c})"
                        )

    def _find_op_identity(self):
        """
        Determine the identity element of the group.

        Returns
        -------
        Any
            The identity element.

        Raises
        ------
        ValueError
            If no identity element exists.
        """
        for e in self.elements:
            if all(
                self._equal(self.op(e, x), x) and self._equal(self.op(x, e), x)
                for x in self.elements
            ):
                self.logger.info(f"Identity found: {e}")
                return e
        raise ValueError("No identity element found")

    def _find_op_inverses(self) -> None:
        """
        Compute left and right inverses for all elements.
        """
        if self.identity is None:
            raise RuntimeError("Identity must be computed before inverses")

        for a in self.elements:
            left_inv = None
            right_inv = None
            for b in self.elements:
                if self._equal(self.op(b, a), self.identity):
                    left_inv = b
                if self._equal(self.op(a, b), self.identity):
                    right_inv = b

            if left_inv is None or right_inv is None:
                raise ValueError(f"No inverse found for element {a}")

            self._left_inverses[a] = left_inv
            self._right_inverses[a] = right_inv

        self.logger.info("All inverses found")

    def _check_abelian(self) -> bool:
        """
        Check whether the group is Abelian (commutative).

        Returns
        -------
        bool
            True if the group is Abelian, False otherwise.
        """
        for a in self.elements:
            for b in self.elements:
                if not self._equal(self.op(a, b), self.op(b, a)):
                    return False
        return True

    def op(
        self,
        a: int | float | Tensor | np.ndarray,
        b: int | float | Tensor | np.ndarray,
    ):
        """
        Apply the group operation.

        Parameters
        ----------
        a, b : group elements

        Returns
        -------
        group element
        """
        return self._op(a, b)

    def inverse(self, element, *, side: str = "right"):
        """
        Return the inverse of an element.

        Parameters
        ----------
        element : group element
            Element whose inverse is requested.
        side : {"right", "left"}, optional
            Whether to return the right or left inverse.
            Defaults to "right".

        Returns
        -------
        group element
            The requested inverse.

        Raises
        ------
        ValueError
            If the element is not in the group or side is invalid.
        """
        if not self._in_elements(element):
            raise ValueError(f"Element {element} is not in the group")

        if side == "right":
            return self._right_inverses[element]
        elif side == "left":
            return self._left_inverses[element]
        else:
            raise ValueError("side must be either 'right' or 'left'")

In [4]:
n = 5
Z = Group(elements=np.arange(n), op=lambda x, y: (x + y) % n)

09:53 AM | __main__ | INFO      | Validating group axioms
09:53 AM | __main__ | INFO      | Identity found: 0
09:53 AM | __main__ | INFO      | All inverses found
09:53 AM | __main__ | SUCCESS   | Group validation successful


In [5]:
text = ""
for i in range(n):
    for j in range(n):
        x = Z.op(i, j)
        text += f"{i} {j} = {x}\n"


In [6]:
print(text)

0 0 = 0
0 1 = 1
0 2 = 2
0 3 = 3
0 4 = 4
1 0 = 1
1 1 = 2
1 2 = 3
1 3 = 4
1 4 = 0
2 0 = 2
2 1 = 3
2 2 = 4
2 3 = 0
2 4 = 1
3 0 = 3
3 1 = 4
3 2 = 0
3 3 = 1
3 4 = 2
4 0 = 4
4 1 = 0
4 2 = 1
4 3 = 2
4 4 = 3



In [7]:
with open("../data/input.txt", "w") as f:
    f.write(text)

## create dataset and dataloaders


In [93]:
# our tokens are '0', '1' .. '4', '='
vocab = [str(_) for _ in range(n)] + ["="]


class GroupTokenizer:
    def __init__(self, vocab: list) -> None:
        self.vocab = vocab
        self.int_to_str = dict(enumerate(sorted(vocab)))
        self.str_to_int = {v: k for k, v in self.int_to_str.items()}

    def decode(self, token_ids: Iterable[int]) -> str:
        tokens = [self.int_to_str[_] for _ in token_ids]
        return " ".join(tokens)

    def encode(self, text: str) -> list:
        tokens = text.strip().split(" ")
        # logger.debug(f"{text = } | {tokens = }")
        return [self.str_to_int[_] for _ in tokens]

In [94]:
class GroupOpDataset(torch.utils.data.Dataset):
    def __init__(self, text: str, tokenizer: GroupTokenizer) -> None:
        super().__init__()
        self.text = text
        self.tokenizer = tokenizer
        self.lines = text.strip().split("\n")
        self.lines = [_ for _ in self.lines if len(_) > 0]

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, index: int) -> Tuple:
        line = self.lines[index]
        token_ids = self.tokenizer.encode(line)
        inputs = token_ids[:-1]
        target = token_ids[1:]
        return (
            torch.tensor(inputs, dtype=torch.long),
            torch.tensor(target, dtype=torch.long),
        )


In [95]:
class GroupOpDataModule(L.LightningDataModule):
    def __init__(self, text: str, vocab: list, batch_size: Optional[int] = 2) -> None:
        super().__init__()
        self.tokenizer = GroupTokenizer(vocab=vocab)
        self.dataset = GroupOpDataset(text, self.tokenizer)
        self.batch_size = batch_size

    def setup(self, stage: str) -> None:
        self.train_dataset, self.val_dataset, self.test_dataset = (
            torch.utils.data.random_split(self.dataset, lengths=[0.3, 0.4, 0.3])
        )

    def train_dataloader(self) -> Any:
        return torch.utils.data.DataLoader(
            self.train_dataset, self.batch_size, shuffle=True, num_workers=15
        )

    def val_dataloader(self) -> Any:
        return torch.utils.data.DataLoader(
            self.val_dataset, self.batch_size, shuffle=False, num_workers=15
        )

    def test_dataloader(self) -> Any:
        return torch.utils.data.DataLoader(
            self.test_dataset, self.batch_size, shuffle=False, num_workers=15
        )

In [96]:
dm = GroupOpDataModule(text=text, vocab=vocab, batch_size=2)
dm.prepare_data()
dm.setup(stage="fit")

In [97]:
dl = dm.train_dataloader()

In [98]:
x, y = next(iter(dl))

In [99]:
x

tensor([[1, 1, 5],
        [0, 4, 5]])

In [100]:
y

tensor([[1, 5, 2],
        [4, 5, 4]])

# Build the model


In [101]:
@dataclass
class Config:
    sequence_length: int = 3
    vocab_size: int = len(vocab)
    embedding_size: int = 128
    head_size: int = 32
    num_heads: int = 4
    hidden_dim: int = 512
    batch_size: int = 2
    grad_accumulation_steps: int = 1
    validation_per_n_steps: int = field(init=False)

    def __post_init__(self):
        self.validation_per_n_steps = 2 * self.batch_size * self.grad_accumulation_steps

In [102]:
config = Config()
logger.debug(f"config.embedding_size = {config.embedding_size}")
logger.debug(f"{config.validation_per_n_steps = }")

10:18 AM | __main__ | DEBUG     | config.embedding_size = 128
10:18 AM | __main__ | DEBUG     | config.validation_per_n_steps = 4


In [103]:
token_embedding = nn.Embedding(config.vocab_size, config.embedding_size)
position_embedding = nn.Embedding(config.sequence_length, config.embedding_size)

tok_emb = token_embedding(x)
pos_emb = position_embedding(torch.arange(x.shape[1]))

In [104]:
x1 = tok_emb + pos_emb
x1.shape

torch.Size([2, 3, 128])

In [105]:
B, S, E = x1.shape


In [106]:
class CausalMultiHeadAttention(nn.Module):
    def __init__(self, config: Config, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.config = config
        self.W_QKV = nn.Linear(
            config.embedding_size, 3 * config.embedding_size, bias=False
        )
        self.W_O = nn.Linear(config.embedding_size, config.embedding_size, bias=False)
        self.register_buffer(
            "mask",
            torch.tril(
                torch.ones(config.sequence_length, config.sequence_length)
                .unsqueeze(0)
                .unsqueeze(0)
            ),
        )

    def forward(self, x):
        B, S, E = x.shape
        qkv = self.W_QKV(x)
        q, k, v = qkv.split(config.embedding_size, dim=-1)  # q, k, v -> (B, S, E)
        # (B, S, E) -> (B, S, NH, HS) where NH * HS = E
        # (B, S, NH, HS) -> (B, NH, S, HS)
        q = q.view(B, S, self.config.num_heads, self.config.head_size).transpose(1, 2)
        k = k.view(B, S, self.config.num_heads, self.config.head_size).transpose(1, 2)
        v = v.view(B, S, self.config.num_heads, self.config.head_size).transpose(1, 2)
        # now compute attention
        sdpa = q @ k.transpose(
            -1, -2
        )  # (B, NH, S, HS) @ (B, NH, HS, S) -> (B, NH, S, S)
        sdpa = sdpa * self.config.head_size**-0.5
        sdpa = sdpa.masked_fill(self.mask[:, :, :S, :S] == 0, -np.inf)  # type: ignore
        # (B, NH, S, S) @ (B, NH, S, HS) -> (B, NH, S, HS)
        sdpa = F.softmax(sdpa, dim=-1) @ v
        # (B, NH, S, HS) -> (B, S, NH, HS) -> (B, S, E)
        sdpa = sdpa.transpose(1, 2).contiguous().view(B, S, self.config.embedding_size)
        mha = self.W_O(sdpa)  # (B, S, E) -> (B, S, E)
        return mha


class MLP(nn.Module):
    def __init__(self, config: Config, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.fc = nn.Linear(config.embedding_size, config.hidden_dim)
        self.gelu = nn.ReLU()
        self.proj = nn.Linear(config.hidden_dim, config.embedding_size)

    def forward(self, x):
        x = self.gelu(self.fc(x))
        x = self.proj(x)
        return x


class AttentionBlock(nn.Module):
    def __init__(self, config: Config, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.config = config
        self.attention = CausalMultiHeadAttention(config)
        self.mlp = MLP(config)

    def forward(self, x: Tensor) -> Tensor:
        x = x + self.attention(x)
        x = x + self.mlp(x)
        return x


class Transformer(nn.Module):
    def __init__(self, config: Config, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.config = Config
        self.token_embeddings = nn.Embedding(config.vocab_size, config.embedding_size)
        self.position_embeddings = nn.Embedding(
            config.sequence_length, config.embedding_size
        )
        self.attention_block = AttentionBlock(config)

    def forward(self, x: Tensor) -> Tensor:
        B, S = x.shape
        assert S <= self.config.sequence_length, (
            f"Sequence length {S} cannot be greater than {self.config.sequence_length}"
        )
        # get the embeddings first
        positions = torch.arange(0, S, dtype=torch.long, device=x.device)
        tok_emb = self.token_embeddings(x)
        pos_emb = self.position_embeddings(positions)
        x = tok_emb + pos_emb
        # forward through each block
        x = self.attention_block(x)
        return x

In [107]:
class GrokkingLM(L.LightningModule):
    def __init__(self, config: Config, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.config = config
        self.transformer = Transformer(self.config)
        self.lm_head = nn.Linear(config.embedding_size, config.vocab_size, bias=False)
        self.criterion = nn.CrossEntropyLoss()

        self.training_loss = []
        self.validation_loss = []

    def forward(self, x: Tensor, target: Optional[Tensor] = None) -> Tensor:
        """Forward method for the model

        Args:
            x (Tensor): the input tensor
            target (Tensor, optional): the target tensor. Defaults to None.

        Returns:
            Tensor: returns the logits if
        """
        B, S = x.shape
        assert S <= self.config.sequence_length, (
            f"Sequence length {S} cannot be greater than {self.config.sequence_length}"
        )
        x = self.transformer(x)
        logits = self.lm_head(x)
        if target is not None:
            logits = logits.view(-1, logits.shape[-1])
            target = target.view(-1)
            loss = self.criterion(logits, target)
            return loss
        return logits

    def training_step(
        self, batch, batch_idx, *args: Any, **kwargs: Any
    ) -> Tensor | Mapping[str, Any] | None:
        x, y = batch
        loss = self(x, y)
        self.training_loss.append(loss.detach().cpu().numpy())
        self.log("train_loss", loss, prog_bar=True, on_step=True)

    def validation_step(
        self, batch, batch_idx, *args: Any, **kwargs: Any
    ) -> Tensor | Mapping[str, Any] | None:
        x, y = batch
        loss = self(x, y)
        self.validation_loss.append(loss.detach().cpu().numpy())
        self.log("val_loss", loss, prog_bar=True, on_epoch=True, on_step=False)

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=1e-3, weight_decay=1)
        return optimizer

In [108]:
model = GrokkingLM(config)

In [109]:
summary(model, input_data=x)

model                                                   #params  w&b_shape                 in shape     out shape  
======================================================  =======  ========================  ===========  ===========
('GrokkingLM', None, None) | 199,168 params                                                                        
├─ transformer: Transformer [1 - 1]                     198,400                            (2, 3)       (2, 3, 128)
│   ├─ token_embeddings: Embedding [2 - 1]              768      w: (6, 128)               (2, 3)       (2, 3, 128)
│   ├─ position_embeddings: Embedding [2 - 2]           384      w: (3, 128)               (3,)         (3, 128)   
│   └─ attention_block: AttentionBlock [2 - 3]          197,248                            (2, 3, 128)  (2, 3, 128)
│       ├─ attention: CausalMultiHeadAttention [3 - 1]  65,536                             (2, 3, 128)  (2, 3, 128)
│       │   ├─ W_QKV: Linear [4 - 1]                    49,152   w: (384

# train the model

In [113]:
wandb_logger = WandbLogger(
    project="grokking", name="test-1", save_dir="../log/", log_model=True
)
csv_logger = CSVLogger(
    save_dir="../log/",
    name="test-1",
    flush_logs_every_n_steps=1,
)

In [114]:
trainer = L.Trainer(
    max_epochs=1000,
    val_check_interval=1.0,
    check_val_every_n_epoch=1,
    log_every_n_steps=2,
    logger=[wandb_logger, csv_logger],
)
trainer.fit(model=model, datamodule=dm)

/home/rudy/code/grokking/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:827: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
wandb: WARNING The anonymous setting has no effect and will be removed in a future version.
wandb: Currently logged in as: roudranil to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



  | Name        | Type             | Params | Mode | FLOPs
----------------------------------------------------------------
0 | transformer | Transformer      | 198 K  | eval | 0    
1 | lm_head     | Linear           | 768    | eval | 0    
2 | criterion   | CrossEntropyLoss | 0      | eval | 0    
----------------------------------------------------------------
199 K     Trainable params
0         Non-trainable params
199 K     Total params
0.797     Total estimated model params size (MB)
0         Modules in train mode
13        Modules in eval mode
0         Total Flops
/home/rudy/code/grokking/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:827: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/rudy/code/grokking/.venv/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:534: Found 14 module(s) in eval mode at the start of training. This may lead to unexpected behavior during training. If this is intentional, you can ignore this warning.


...


Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

/home/rudy/code/grokking/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3709: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.


Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7ff526362090>> (for post_run_cell), with arguments args (<ExecutionResult object at 7ff527378d10, execution_count=114 error_before_exec=None error_in_exec=1 info=<ExecutionInfo object at 7ff52737b210, raw_cell="trainer = L.Trainer(
    max_epochs=1000,
    val_.." transformed_cell="trainer = L.Trainer(
    max_epochs=1000,
    val_.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/rudy/code/grokking/notebooks/grokking.ipynb#X40sZmlsZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [111]:
min(len(dm.train_dataloader()), config.validation_per_n_steps)

4